In [0]:
from scipy import io
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
import sklearn.svm

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading DataSet
### Dataset include image and a ground truth

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/IndianPines/Indian_pines_corrected.mat')
image = loaded_dataset['indian_pines_corrected']

In [0]:
ground_truth = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/IndianPines/Indian_pines_gt.mat')
ground_truth = ground_truth['indian_pines_gt']

### Class Labels

In [0]:
label_values = ["Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]

### Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

### Visualizing data in pandas

In [0]:
data = pd.DataFrame(final_output)

In [42]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
0,3172,4142,4506,4279,4782,5048,5213,5106,5053,4750,4816,4769,4610,4805,4828,4861,4767,4624,4549,4463,4462,4446,4445,4336,4381,4319,4207,4305,4311,3991,4168,3942,4061,4362,4318,4252,4869,5284,5055,3591,...,1396,1381,1396,1381,1353,1346,1341,1332,1324,1310,1318,1330,1310,1292,1280,1275,1266,1264,1233,1241,1232,1215,1215,1187,1168,1171,1150,1134,1123,1135,1094,1090,1112,1090,1062,1069,1057,1020,1020,3
1,2580,4266,4502,4426,4853,5249,5352,5353,5347,5065,5141,5100,4994,5172,5290,5289,5217,5053,5033,4939,4931,4941,4902,4824,4859,4805,4698,4794,4806,4452,4628,4433,4643,4967,4853,4760,5449,5768,5684,3987,...,1421,1415,1428,1415,1379,1370,1360,1353,1352,1336,1346,1351,1330,1315,1305,1292,1282,1286,1259,1259,1250,1229,1232,1195,1177,1184,1153,1137,1138,1137,1108,1104,1117,1091,1079,1085,1064,1029,1020,3
2,3687,4266,4421,4498,5019,5293,5438,5427,5383,5132,5227,5172,5097,5313,5411,5412,5341,5191,5140,5069,5110,5119,5046,4981,5023,4987,4862,4965,4992,4595,4756,4529,4801,5077,4983,4868,5515,5972,5913,4027,...,1446,1440,1443,1425,1390,1379,1376,1363,1355,1347,1361,1356,1341,1330,1321,1304,1290,1289,1263,1269,1261,1245,1241,1214,1185,1188,1156,1147,1149,1144,1111,1114,1114,1100,1065,1092,1061,1030,1016,3
3,2749,4258,4603,4493,4958,5234,5417,5355,5349,5096,5147,5078,5040,5237,5321,5344,5255,5121,5035,4956,4994,4980,4905,4857,4900,4831,4720,4848,4847,4484,4613,4402,4674,4966,4848,4776,5473,5894,5789,4086,...,1432,1427,1426,1416,1386,1374,1375,1359,1343,1343,1354,1351,1333,1329,1313,1296,1280,1281,1251,1255,1253,1238,1223,1207,1188,1188,1154,1143,1144,1146,1122,1108,1109,1109,1071,1088,1060,1030,1006,3
4,2746,4018,4675,4417,4886,5117,5215,5096,5098,4834,4853,4857,4734,4879,4976,4958,4885,4754,4647,4532,4586,4591,4492,4453,4497,4398,4297,4408,4401,4102,4227,4075,4264,4529,4490,4438,5001,5378,5321,3779,...,1401,1397,1395,1390,1368,1349,1354,1340,1330,1324,1336,1332,1320,1307,1287,1283,1267,1265,1239,1240,1239,1229,1212,1202,1178,1178,1143,1135,1138,1135,1110,1107,1112,1094,1072,1087,1052,1034,1019,3


### Droping the row if ground_truth value is zero

In [43]:
print("Percentage of column which will be droped",(data.size - data[data[200] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.48746730083234247 %


In [0]:
data = data[data[200] != 0]

### Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

### Feature Selection


In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

X = SelectKBest(chi2, k=150).fit_transform(X, y)

### OneHotEncoding the target column

In [0]:
# onehotencoder = OneHotEncoder() 
# y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray()

### Standardizing the feature columns

In [0]:
X = preprocessing.scale(X)

### Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=25)
# X_train = X
# y_train = y

# Preparing SVM model

### Defining grid parameters

In [0]:
# SVM_GRID_PARAMS = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]},
#                    {'kernel': ['poly'], 'degree': [3], 'gamma': [1e-1, 1e-2, 1e-3]}]

# Best Parameters
SVM_GRID_PARAMS = [{'kernel': ['rbf'], 'gamma': [1e-2],
                    'C': [100]}]

### Defining the weight 

In [0]:
class_weight = None

### Defining the model

Link to learn more about svm https://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use

In [0]:
model = sklearn.svm.SVC(class_weight=class_weight)

### Applying the grid search

In [0]:
model = sklearn.model_selection.GridSearchCV(model, SVM_GRID_PARAMS, verbose=5, n_jobs=4)

### Model fitting

In [54]:
model.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:    7.0s remaining:   10.5s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   10.1s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=4,
             param_grid=[{'C': [100], 'gamma': [0.01], 'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=5)

### Best parameters

In [0]:
# print("SVM best parameters : {}".format(model.best_params_))

### Accuracy of training set

In [56]:
model.score(X_train, y_train)

0.9824365765263451

### Cross Validation using kfold

In [0]:
kfold = KFold(n_splits=2)

In [58]:
result = cross_val_score(model, X_test, y_test, cv=kfold, scoring='accuracy')

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.0s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.9s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.0s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.9s finished


In [59]:
result.mean()

0.8637407748023822

### Accuracy of testing set

In [0]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,cohen_kappa_score
prediction = model.predict(X_test)

In [61]:
accuracy_score(y_test, prediction)

0.9313821138211382

In [62]:
precision_score(y_test, prediction, average="weighted")

0.9316135810812622

In [63]:
recall_score(y_test, prediction, average="weighted")

0.9313821138211382

In [64]:
f1_score(y_test, prediction, average="weighted")

0.9312842400308988

In [65]:
cohen_kappa_score(y_test, prediction, labels=list(x for x in range(1,17)), weights=None, sample_weight=None)

0.9214869517624399

In [66]:
print(classification_report(y_test, prediction, target_names=label_values))

                              precision    recall  f1-score   support

                     Alfalfa       1.00      0.92      0.96        13
                 Corn-notill       0.93      0.90      0.92       424
                Corn-mintill       0.93      0.89      0.91       228
                        Corn       0.85      0.86      0.86        79
               Grass-pasture       0.99      0.95      0.97       128
                 Grass-trees       0.96      0.97      0.96       240
         Grass-pasture-mowed       1.00      0.75      0.86         4
               Hay-windrowed       0.99      1.00      1.00       136
                        Oats       0.86      1.00      0.92         6
              Soybean-notill       0.88      0.91      0.89       269
             Soybean-mintill       0.92      0.94      0.93       744
               Soybean-clean       0.94      0.94      0.94       181
                       Wheat       1.00      0.98      0.99        60
                   